In [0]:
# Importing modules
from keras.applications import VGG16
from keras import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator, load_img
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Creating data generators from file directory


In [0]:
# Paths to directories
train_path = "/content/drive/My Drive/ml/train"
val_path = "/content/drive/My Drive/ml/val"
test_path = "/content/drive/My Drive/ml/"

Creating data generators



In [0]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, classes=['hate', 'nohate'], batch_size=50, 
                                                         target_size=(224, 224))
val_batches = ImageDataGenerator().flow_from_directory(val_path, classes=['hate', 'nohate'], batch_size=50, 
                                                       target_size=(224, 224))
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), batch_size=50,
                                                        class_mode=None, classes=['test'])

Found 8500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Found 1000 images belonging to 1 classes.


In [0]:
# Loading in the vgg16 model
vgg16_mod = VGG16()

553467904/553467096 [==============================] - 16s 0us/step


In [0]:
model = Sequential()
for layer in vgg16_mod.layers:
    model.add(layer)

In [0]:
model.layers.pop()


Freezing all the layers


In [0]:
for layer in model.layers:
    layer.trainable = False

Output of two for either hate or nohate speech


In [0]:
model.add(Dense(2, activation='softmax'))


Compiling model with loss equal to categorical crossentropy


In [0]:
model.compile("adam", loss='categorical_crossentropy', metrics=['accuracy'])


In [0]:
model.fit_generator(train_batches, steps_per_epoch=170, validation_data=val_batches, validation_steps=10, epochs=10,
                    verbose=2)

Epoch 1/10
 - 130s - loss: 0.6415 - accuracy: 0.6412 - val_loss: 0.6075 - val_accuracy: 0.7500
Epoch 2/10
 - 128s - loss: 0.6400 - accuracy: 0.6412 - val_loss: 0.5246 - val_accuracy: 0.7500
Epoch 3/10
 - 128s - loss: 0.6386 - accuracy: 0.6412 - val_loss: 0.5734 - val_accuracy: 0.7500
Epoch 4/10
 - 128s - loss: 0.6374 - accuracy: 0.6411 - val_loss: 0.6276 - val_accuracy: 0.7500
Epoch 5/10
 - 128s - loss: 0.6363 - accuracy: 0.6412 - val_loss: 0.6045 - val_accuracy: 0.7500
Epoch 6/10
 - 127s - loss: 0.6353 - accuracy: 0.6412 - val_loss: 0.6184 - val_accuracy: 0.7520
Epoch 7/10
 - 127s - loss: 0.6344 - accuracy: 0.6412 - val_loss: 0.5514 - val_accuracy: 0.7520
Epoch 8/10
 - 127s - loss: 0.6335 - accuracy: 0.6411 - val_loss: 0.5651 - val_accuracy: 0.7520
Epoch 9/10
 - 127s - loss: 0.6327 - accuracy: 0.6407 - val_loss: 0.5689 - val_accuracy: 0.7500
Epoch 10/10
 - 127s - loss: 0.6319 - accuracy: 0.6412 - val_loss: 0.5511 - val_accuracy: 0.7500


In [0]:
predictions = model.predict_generator(test_batches, steps=20)

In [0]:
predictions

array([[0.34703425, 0.6529658 ],
       [0.27513424, 0.7248658 ],
       [0.29582274, 0.70417726],
       ...,
       [0.2840645 , 0.7159355 ],
       [0.4096922 , 0.59030783],
       [0.34754282, 0.6524572 ]], dtype=float32)

In [0]:
prob_hate = predictions[:, 0]
class_hate = np.where(prob_hate > .5, 1, 0)

In [0]:
ids = pd.read_csv("/content/drive/My Drive/ml/submission_format.csv")

In [0]:
ids_ = ids['id']

In [0]:
ids_initial = ids_

In [0]:
initial_df = pd.DataFrame({"id": ids_initial, "proba": prob_hate, "label": class_hate}).set_index('id')

In [0]:
final_csv = initial_df.reindex(ids_).reset_index()

In [0]:
final_csv.head()

,id,proba,label
0,16395,0.347034,0
1,37405,0.275134,0
2,94180,0.295823,0
3,54321,0.411970,0
4,97015,0.333861,0


In [0]:
final_csv.to_csv("/content/drive/My Drive/ml/vgg_pred_partial.csv", index=False)

In [0]:
model.save('/content/drive/My Drive/ml/vgg_second.h5')